In [ ]:
%env PYTHONPATH=

In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.12-Linux-x86_64.sh
 MINICONDA_PREFIX=/usr/local 
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT 
chmod +x $MINICONDA_INSTALLER_SCRIPT 
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
%%bash
conda install --channel defaults conda python=3.7 --yes 
conda update --channel defaults --all --yes

In [ ]:
import sys
sys.path

In [5]:
import sys
_ = (sys.path
        .append("/usr/local/lib/python3.7/site-packages"))

In [6]:
!conda install -c conda-forge cyvlfeat

Solving environment: - \ | / - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cyvlfeat


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2022.4.26-~ --> conda-forge::ca-certificates-2021.10.8-ha878542_0
  certifi            pkgs/main::certifi-2021.10.8-py37h06a~ --> conda-forge::certifi-2021.10.8-py37h89c1867_2
  conda              pkgs/main::conda-4.12.0-py37h06a4308_0 --> conda-forge::conda-4.12.0-py37h89c1867_0


Proceed ([y]/n)? y

Preparing transaction: | done
Verifying transaction: - done
Executing transaction: | done


In [7]:
import torch
import os
import cv2
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import MiniBatchKMeans
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from glob import glob
import cyvlfeat
from tqdm import tqdm

In [8]:
#load data
train_data_base_path = "/content/drive/MyDrive/training"
test_data_base_path = "/content/drive/MyDrive/testing"
train_X = []
train_y = []
test=[]
categorys = os.listdir(train_data_base_path)
if categorys.__contains__('.DS_Store'):
    categorys.remove('.DS_Store')
for category in categorys:
  train_data_path = glob(os.path.join(train_data_base_path, category, '*.jpg'))
  for file in tqdm(train_data_path):
    image = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    #
    image = (image - np.mean(image))/np.std(image)
    #
    image = (image-np.min(image))/(np.max(image)-np.min(image))

    train_X.append(image)
    train_y.append(category)
for file in glob(os.path.join(test_data_base_path, '*.jpg')):
  image = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
  test.append(image)
#encode label
le = preprocessing.LabelEncoder()
le.fit(train_y)
train_y = le.transform(train_y)

100%|██████████| 100/100 [00:00<00:00, 169.34it/s]


In [9]:
print(train_X[0:3])
print(train_y[0:3])

[array([[0.80237154, 0.51383399, 0.22529644, ..., 0.40316206, 0.27667984,
        0.33201581],
       [0.38735178, 0.27272727, 0.16205534, ..., 0.49407115, 0.46640316,
        0.52173913],
       [0.12252964, 0.12252964, 0.12648221, ..., 0.39920949, 0.46640316,
        0.51383399],
       ...,
       [0.97233202, 0.96442688, 0.94466403, ..., 0.96047431, 0.95652174,
        0.95652174],
       [0.96442688, 0.95652174, 0.94466403, ..., 0.95256917, 0.95652174,
        0.95652174],
       [0.94466403, 0.94466403, 0.94466403, ..., 0.9486166 , 0.95652174,
        0.95652174]]), array([[0.80314961, 0.80314961, 0.80314961, ..., 0.96062992, 0.96062992,
        0.96062992],
       [0.80314961, 0.80314961, 0.80708661, ..., 0.96062992, 0.96062992,
        0.96062992],
       [0.80314961, 0.80708661, 0.80708661, ..., 0.96062992, 0.96062992,
        0.96062992],
       ...,
       [0.59448819, 0.57874016, 0.61023622, ..., 0.62992126, 0.5984252 ,
        0.58661417],
       [0.61417323, 0.59055118, 0

In [10]:
#get sift dfit phow feature
def get_sift_feature(X,sift_type = "dsift"):
  num = 0
  features = []
  if sift_type == "sift":
    feature_generate = cyvlfeat.sift.sift
  elif sift_type == "dsift":
    feature_generate = cyvlfeat.sift.dsift
  elif sift_type == "phow":
    feature_generate = cyvlfeat.sift.phow
  for image in tqdm(X):
    if sift_type == "sift":
      feature = feature_generate(image)
    else:
      _,feature = feature_generate(image,step = 4)
    features.append(feature) 
  return features

In [11]:
#kmeans 
def get_kmeans_centers(feature):
  # creat feature array
  for i in range(0,len(feature)):
    if i == 0:
      feature_new = feature[i]
    else:
      feature_new = np.append(feature_new,feature[i],axis = 0)
  #set kmeans
  kmeans = MiniBatchKMeans(n_clusters=500)       
  kmeans.fit(feature_new)
  return kmeans 

In [12]:
#get hog feature
def get_hog_feature(kmeans,feature): 
  hog_feature = []
  for i in tqdm(range(0,len(feature))):
    vocab = np.zeros(len(kmeans.cluster_centers_))
    res=kmeans.predict(feature[i])
    for data in res:
      vocab[data] +=1 
    hog_feature.append(vocab)
  return hog_feature

In [24]:
def sift_hog_classifier(X,y,classifier = "SVM"):
  X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.3)
  if  classifier == "SVM":
    clf = SVC(C=30)
  elif classifier == "RFC":
    clf = RandomForestClassifier()
  elif classifier == "LR":
    clf = LogisticRegression(C=10)
  elif classifier == "MLP":
    clf = KNeighborsClassifier()
  clf = clf.fit(X_train,y_train)
  y_hat_train = clf.predict(X_train)
  y_hat_val = clf.predict(X_val)
  print("--------------"+classifier+"--------------")
  print("train accuracy:"+str(accuracy_score(y_train,y_hat_train)))
  print("val accuracy:"+str(accuracy_score(y_val,y_hat_val)))


In [14]:
sift_feature = get_sift_feature(train_X,"sift")
dsift_feature = get_sift_feature(train_X,"dsift")
phow_feature = get_sift_feature(train_X,"phow")
print(phow_feature[0:10])

100%|██████████| 1500/1500 [01:55<00:00, 13.03it/s]

[array([[ 27,  32,  16, ...,  13,  37,  34],
       [ 32,  40,  25, ...,  63,  46,   4],
       [  9,   3,  10, ...,  21,  23,   6],
       ...,
       [  9,  40, 140, ...,   6,  12,   1],
       [  8,  40, 150, ...,   6,  12,   2],
       [  5,  24, 147, ...,   6,  10,   3]], dtype=uint8), array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,  59, ...,   5,   1,   1],
       [  0,   0, 113, ...,  10,   0,   1],
       [  0,   0, 129, ...,  17,   0,   0]], dtype=uint8), array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 1,  7, 45, ...,  7,  7,  0],
       [ 0,  3, 28, ..., 13, 17,  0],
       [ 0,  0, 18, ..., 18, 30,  0]], dtype=uint8), array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 1,  1,  7, ...,  1,  0,  5]

In [15]:
import gc

kmeans_sift = get_kmeans_centers(sift_feature)
hog_sift_feature = get_hog_feature(kmeans_sift,sift_feature)
del kmeans_sift
gc.collect()
kmeans_dsift = get_kmeans_centers(dsift_feature)
hog_dsift_feature =get_hog_feature(kmeans_dsift,dsift_feature)
del kmeans_dsift
gc.collect()
kmeans_phow = get_kmeans_centers(phow_feature)
hog_phow_feature =get_hog_feature(kmeans_phow,phow_feature)
del kmeans_phow
gc.collect()

100%|██████████| 1500/1500 [00:26<00:00, 56.38it/s]


0

In [17]:
# the feature extracted by resNet
training_features = np.load("/content/drive/MyDrive/training_features.npy")
training_labels = np.load("/content/drive/MyDrive/training_labels.npy")

In [20]:
sift_hog_classifier(hog_sift_feature,train_y,classifier = "SVM")
sift_hog_classifier(hog_dsift_feature,train_y,classifier = "SVM")
sift_hog_classifier(hog_phow_feature,train_y,classifier = "SVM")

sift_hog_classifier(training_features,training_labels,classifier = "SVM")


--------------SVM--------------
train accuracy:1.0
val accuracy:0.29777777777777775
--------------SVM--------------
train accuracy:1.0
val accuracy:0.6933333333333334
--------------SVM--------------
train accuracy:1.0
val accuracy:0.6


In [21]:
sift_hog_classifier(hog_sift_feature,train_y,classifier = "RFC")
sift_hog_classifier(hog_dsift_feature,train_y,classifier = "RFC")
sift_hog_classifier(hog_phow_feature,train_y,classifier = "RFC")

sift_hog_classifier(training_features,training_labels,classifier = "RFC")

--------------RFC--------------
train accuracy:1.0
val accuracy:0.24444444444444444
--------------RFC--------------
train accuracy:1.0
val accuracy:0.68
--------------RFC--------------
train accuracy:1.0
val accuracy:0.5111111111111111


In [22]:
sift_hog_classifier(hog_sift_feature,train_y,classifier = "LR")
sift_hog_classifier(hog_dsift_feature,train_y,classifier = "LR")
sift_hog_classifier(hog_phow_feature,train_y,classifier = "LR")

sift_hog_classifier(training_features,training_labels,classifier = "LR")

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


--------------LR--------------
train accuracy:1.0
val accuracy:0.20666666666666667


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


--------------LR--------------
train accuracy:1.0
val accuracy:0.6755555555555556
--------------LR--------------
train accuracy:1.0
val accuracy:0.5638888888888889


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [25]:
sift_hog_classifier(hog_sift_feature,train_y,classifier = "MLP")
sift_hog_classifier(hog_dsift_feature,train_y,classifier = "MLP")
sift_hog_classifier(hog_phow_feature,train_y,classifier = "MLP")

sift_hog_classifier(training_features,training_labels,classifier = "MLP")

--------------MLP--------------
train accuracy:0.2542857142857143
val accuracy:0.16444444444444445
--------------MLP--------------
train accuracy:0.6847619047619048
val accuracy:0.52
--------------MLP--------------
train accuracy:0.6130952380952381
val accuracy:0.3638888888888889
